In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/erich/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

In [2]:
# preparing the data
# import the data, data is stacked
movie_ratings_raw = pd.read_csv('/home/erich/Downloads/Exercises/movie_smallset.csv')

user_indices = [i-1 for i in movie_ratings_raw.user_id.values]
item_indices = [i-1 for i in movie_ratings_raw.user_id.values]
R_known = movie_ratings_raw.values

In [3]:
movie_ratings_pt = movie_ratings_raw.pivot(index = 'user_id', columns = 'item_id', values = 'rating').fillna(0)

#Unpivot table to name columns and rows, for clarification and visualization of data.
index = movie_ratings_pt.index.union(movie_ratings_pt.columns)
movie_ratings = movie_ratings_pt.reindex(index=index, columns=index, fill_value=0)
movie_ratings.columns = ['Toy Story','Terminator','Terminator 2','Lion King','Despicable Me','Despicable Me 2',
                                       'Die Hard', 'Die Hard 2','Toy Story 2','Die Hard 3']
movie_ratings.index = ['Justin','Mike','Stef','Jim','Claire','Joe','Amie','Charles','Katie','Chuck']
movie_ratings

,Toy Story,Terminator,Terminator 2,Lion King,Despicable Me,Despicable Me 2,Die Hard,Die Hard 2,Toy Story 2,Die Hard 3
Justin,1.0,5.0,0.0,1.0,2.0,1.0,5.0,5.0,0.0,4.0
Mike,2.0,4.0,5.0,0.0,1.0,1.0,5.0,0.0,1.0,5.0
Stef,0.0,5.0,4.0,1.0,0.0,1.0,5.0,4.0,1.0,5.0
Jim,1.0,5.0,0.0,1.0,1.0,1.0,5.0,0.0,1.0,5.0
Claire,1.0,0.0,5.0,2.0,1.0,2.0,4.0,5.0,0.0,5.0
Joe,5.0,1.0,1.0,0.0,5.0,5.0,1.0,1.0,5.0,1.0
Amie,5.0,0.0,1.0,5.0,5.0,5.0,1.0,1.0,5.0,0.0
Charles,0.0,1.0,1.0,5.0,5.0,5.0,1.0,1.0,5.0,1.0
Katie,5.0,1.0,1.0,0.0,5.0,5.0,1.0,1.0,5.0,1.0
Chuck,5.0,1.0,0.0,5.0,5.0,5.0,1.0,1.0,5.0,1.0


In [4]:
# Prepare data
R = np.array(movie_ratings)
R_known = movie_ratings_raw.rating.values
N = 10    # number of users
M = 10    # number of items
K = 2     # number of hidden features
p = np.random.rand(N,K)
q = np.random.rand(K,M)

In [5]:
# Prediction
P = tf.Variable(p, dtype=tf.float32, name='P')
Q = tf.Variable(q, dtype=tf.float32, name='Q')
R_hat = tf.matmul(P, Q)

# For cost function, we want to compare only the known values
R_hat_flat = tf.reshape(R_hat, [-1])
R_hat_known = tf.gather(R_hat_flat, user_indices * tf.shape(R_hat)[1] + 
              item_indices, name='extracting_user_rate')

In [6]:
# Cost
diff_ratings = tf.subtract(R_hat_known, R_known, name='diff_ratings')
diff_ratings_abs = tf.abs(diff_ratings, name="diff_ratings_abs")
base_cost = tf.reduce_sum(diff_ratings_abs, name="sum_abs_error")

In [7]:
# Regularization
lda = tf.constant(.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.abs(P, name='user_abs'), name='user_norm'),
                   tf.reduce_sum(tf.abs(Q, name='item_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [8]:
# Total Cost
cost = tf.add(base_cost, regularizer)

In [9]:
# Create an optimizer to minimize the loss
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(cost)

In [10]:
# Execute the TF session
sess = tf.Session();
init = tf.global_variables_initializer()
sess.run(init)

for i in range(5000):
    sess.run(train)

In [11]:
# Accuracy for known R ratings vs predicted R_hat ratings
diff_ratings = tf.subtract(R_hat_known, R_known, name='diff_ratings')
diff_ratings_abs = tf.abs(diff_ratings, name="diff_ratings_abs")
sum_diff_ratings_abs = tf.reduce_sum(diff_ratings_abs, name="sum_diff_ratings_abs")
accuracy = tf.div(sum_diff_ratings_abs, R_known.size, name="accuracy")
print(sess.run(accuracy))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
1.6810653


In [12]:
# Visualize known R ratings, to compare with R_hat.
movie_ratings

,Toy Story,Terminator,Terminator 2,Lion King,Despicable Me,Despicable Me 2,Die Hard,Die Hard 2,Toy Story 2,Die Hard 3
Justin,1.0,5.0,0.0,1.0,2.0,1.0,5.0,5.0,0.0,4.0
Mike,2.0,4.0,5.0,0.0,1.0,1.0,5.0,0.0,1.0,5.0
Stef,0.0,5.0,4.0,1.0,0.0,1.0,5.0,4.0,1.0,5.0
Jim,1.0,5.0,0.0,1.0,1.0,1.0,5.0,0.0,1.0,5.0
Claire,1.0,0.0,5.0,2.0,1.0,2.0,4.0,5.0,0.0,5.0
Joe,5.0,1.0,1.0,0.0,5.0,5.0,1.0,1.0,5.0,1.0
Amie,5.0,0.0,1.0,5.0,5.0,5.0,1.0,1.0,5.0,0.0
Charles,0.0,1.0,1.0,5.0,5.0,5.0,1.0,1.0,5.0,1.0
Katie,5.0,1.0,1.0,0.0,5.0,5.0,1.0,1.0,5.0,1.0
Chuck,5.0,1.0,0.0,5.0,5.0,5.0,1.0,1.0,5.0,1.0


In [13]:
# Visualize R_hat rating predictions, to compare with known R.
pred = sess.run(R_hat)
pred_df = pd.DataFrame(pred)
pred_df

,0,1,2,3,4,5,6,7,8,9
0,2.000952,1.890421,2.631684,1.377647,1.947614,1.583759,3.051883,1.302450,1.521465,2.969096
1,2.030813,2.003105,2.578344,1.423817,2.098240,1.542459,2.998754,1.500119,1.604518,3.266509
2,2.978850,2.748114,3.990394,2.030857,2.804200,2.408643,4.620697,1.799331,2.217746,4.221828
3,1.433893,1.404498,1.831267,1.002332,1.467353,1.096640,2.128811,1.038443,1.125876,2.276921
4,1.870418,1.899400,2.314953,1.327888,2.010949,1.378741,2.698251,1.496632,1.516727,3.171818
5,1.256862,1.169913,1.672254,0.860032,1.198146,1.008278,1.937451,0.781143,0.943164,1.812487
6,3.152252,2.841447,4.295737,2.128871,2.871541,2.600076,4.967509,1.763472,2.299238,4.267921
7,1.078450,1.163434,1.259910,0.786337,1.257726,0.742477,1.476016,1.006983,0.923293,2.033380
8,1.362232,1.262526,1.818444,0.930475,1.290725,1.097011,2.106266,0.835094,1.018329,1.948054
9,2.676064,2.842669,3.174876,1.937792,3.057207,1.876414,3.714297,2.405312,2.259428,4.912970
